In [1]:
import os, re, json, time
from typing import List, Dict
import pandas as pd
from tqdm import tqdm
from tenacity import retry, wait_exponential_jitter, stop_after_attempt, retry_if_exception_type

from google.cloud import vision
from google.api_core.exceptions import ResourceExhausted, DeadlineExceeded, ServiceUnavailable, GoogleAPICallError

In [3]:
df = pd.read_csv('apartments_with_neighborhood.csv')
df

,listing_id,name,address,price,bedrooms,bathrooms,sqft,url,image_url,zip,neighborhood_group
0,apt_1,The Summit at The Reserve,"725 University Blvd, Round Rock, TX 78665","$1,399 - $2,800",1-2 Beds,NaN,NaN,https://www.apartments.com/the-summit-at-the-r...,https://images1.apartments.com/i2/Xdm0jW8F5j9X...,78665,Round Rock / Pflugerville / Hutto
1,apt_2,Luxe of Buda,"1664 Hillside Ter, Buda, TX 78610","$1,297 - $7,599",1-4 Beds,NaN,NaN,https://www.apartments.com/luxe-of-buda-buda-t...,https://images1.apartments.com/i2/Pj5KKd536h1m...,78610,Buda / Kyle / Creedmoor
2,apt_3,Del Via Apartments,"11600 Spirit Dr, Del Valle, TX 78617","$1,185 - $2,355",Studio - 3 Beds,NaN,NaN,https://www.apartments.com/del-via-apartments-...,https://images1.apartments.com/i2/ZOMQeQnrNgzt...,78617,Bastrop / Del Valle / Manor
3,apt_4,Woodview,"691 Union Chapel Rd, Cedar Creek, TX 78612","$1,329 - $2,259",Studio - 3 Beds,NaN,NaN,https://www.apartments.com/woodview-cedar-cree...,https://images1.apartments.com/i2/d86O5UCtOc5C...,78612,Bastrop / Del Valle / Manor
4,apt_5,Ellwood at Lake Travis,"7655 N Ranch Road 620, Austin, TX 78726","$995 - $3,085",1-3 Beds,NaN,NaN,https://www.apartments.com/ellwood-at-lake-tra...,https://images1.apartments.com/i2/Eb86_um5jntA...,78726,Northwest Austin / Lake Travis
...,...,...,...,...,...,...,...,...,...,...,...
145,apt_146,Lakewood,"14200 The Lakes Blvd, Pflugerville, TX 78660","$911 - $1,961",1-2 Beds,NaN,NaN,https://www.apartments.com/lakewood-pflugervil...,https://images1.apartments.com/i2/yRyFZU5eKwtM...,78660,Round Rock / Pflugerville / Hutto
146,apt_147,Hudson Miramont,"8818 Travis Hills Dr, Austin, TX 78735","$1,325 - $2,199",1-3 Beds,NaN,NaN,https://www.apartments.com/hudson-miramont-aus...,https://images1.apartments.com/i2/-UxuVhyasz0T...,78735,Other
147,apt_148,The Aidan Apartments,"210 S Amberwood St, Kyle, TX 78640","$941 - $4,710",1-2 Beds,NaN,NaN,https://www.apartments.com/the-aidan-apartment...,https://images1.apartments.com/i2/PQSjyUixas03...,78640,Buda / Kyle / Creedmoor
148,apt_149,Echo,"4527 N Lamar Blvd, Austin, TX 78751","$1,585 - $2,837",1-2 Beds,NaN,NaN,https://www.apartments.com/echo-austin-tx/l5j0...,https://images1.apartments.com/i2/WrRSgP86LODN...,78751,Other


In [5]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"[redacted].json"

from google.cloud import vision
client = vision.ImageAnnotatorClient()
print("✅ Connected successfully!")



✅ Connected successfully!


In [7]:
import re
from google.cloud import vision
from tqdm import tqdm

client = vision.ImageAnnotatorClient()

def five_words_from_labels(url):
    try:
        img = vision.Image()
        img.source.image_uri = url
        resp = client.label_detection(image=img, max_results=20)
        labels = [l.description.lower().strip() for l in resp.label_annotations]
        # harvest distinct word tokens until we have 5
        tokens, seen = [], set()
        for phrase in labels:
            for w in phrase.split():
                w = re.sub(r"[^a-z]+", "", w.lower())
                if not w or w in seen:
                    continue
                seen.add(w)
                tokens.append(w)
                if len(tokens) == 5:
                    return " ".join(tokens)
        return " ".join(tokens[:5])
    except Exception as e:
        # If an image 404s or other transient error, return blank so you can retry later
        return ""

# Apply with progress bar
tqdm.pandas(desc="Processing images")
df["image_labels_5words"] = df["image_url"].progress_apply(five_words_from_labels)

df.to_csv("apartments_with_neighborhood_with_image_labels.csv", index=False)
print("✅ Saved apartments_with_neighborhood_with_image_labels.csv")


Processing images: 100%|████████████████████████████| 150/150 [01:18<00:00,  1.91it/s]

✅ Saved apartments_with_neighborhood_with_image_labels.csv


In [9]:
df1 = pd.read_csv('apartments_with_neighborhood_with_image_labels.csv')
df1

,listing_id,name,address,price,bedrooms,bathrooms,sqft,url,image_url,zip,neighborhood_group,image_labels_5words
0,apt_1,The Summit at The Reserve,"725 University Blvd, Round Rock, TX 78665","$1,399 - $2,800",1-2 Beds,NaN,NaN,https://www.apartments.com/the-summit-at-the-r...,https://images1.apartments.com/i2/Xdm0jW8F5j9X...,78665,Round Rock / Pflugerville / Hutto,facade architecture apartment engineering mixe...
1,apt_2,Luxe of Buda,"1664 Hillside Ter, Buda, TX 78610","$1,297 - $7,599",1-4 Beds,NaN,NaN,https://www.apartments.com/luxe-of-buda-buda-t...,https://images1.apartments.com/i2/Pj5KKd536h1m...,78610,Buda / Kyle / Creedmoor,interior design flooring floor wood
2,apt_3,Del Via Apartments,"11600 Spirit Dr, Del Valle, TX 78617","$1,185 - $2,355",Studio - 3 Beds,NaN,NaN,https://www.apartments.com/del-via-apartments-...,https://images1.apartments.com/i2/ZOMQeQnrNgzt...,78617,Bastrop / Del Valle / Manor,interior design apartment furniture flooring
3,apt_4,Woodview,"691 Union Chapel Rd, Cedar Creek, TX 78612","$1,329 - $2,259",Studio - 3 Beds,NaN,NaN,https://www.apartments.com/woodview-cedar-cree...,https://images1.apartments.com/i2/d86O5UCtOc5C...,78612,Bastrop / Del Valle / Manor,countertop flooring interior design sink
4,apt_5,Ellwood at Lake Travis,"7655 N Ranch Road 620, Austin, TX 78726","$995 - $3,085",1-3 Beds,NaN,NaN,https://www.apartments.com/ellwood-at-lake-tra...,https://images1.apartments.com/i2/Eb86_um5jntA...,78726,Northwest Austin / Lake Travis,furniture wood flooring interior design
...,...,...,...,...,...,...,...,...,...,...,...,...
145,apt_146,Lakewood,"14200 The Lakes Blvd, Pflugerville, TX 78660","$911 - $1,961",1-2 Beds,NaN,NaN,https://www.apartments.com/lakewood-pflugervil...,https://images1.apartments.com/i2/yRyFZU5eKwtM...,78660,Round Rock / Pflugerville / Hutto,groundcover birdseye view aerial photography
146,apt_147,Hudson Miramont,"8818 Travis Hills Dr, Austin, TX 78735","$1,325 - $2,199",1-3 Beds,NaN,NaN,https://www.apartments.com/hudson-miramont-aus...,https://images1.apartments.com/i2/-UxuVhyasz0T...,78735,Other,swimming pool resort villa hotel
147,apt_148,The Aidan Apartments,"210 S Amberwood St, Kyle, TX 78640","$941 - $4,710",1-2 Beds,NaN,NaN,https://www.apartments.com/the-aidan-apartment...,https://images1.apartments.com/i2/PQSjyUixas03...,78640,Buda / Kyle / Creedmoor,apartment composite material concrete aluminium
148,apt_149,Echo,"4527 N Lamar Blvd, Austin, TX 78751","$1,585 - $2,837",1-2 Beds,NaN,NaN,https://www.apartments.com/echo-austin-tx/l5j0...,https://images1.apartments.com/i2/WrRSgP86LODN...,78751,Other,swimming pool


In [11]:
df1['image_labels_5words'].isna().sum()

15